In [1]:
import json
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.append('/home/jkuettel/NLP_spark/src')
sys.path.append('/home/jkuettel/NLP_spark')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus, Sent_Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, Sent_Inter_Annotator_Agreement, _get_score_article, row_to_span_list 

from definitions import ROOT_DIR
from pyannote.core import Segment
from definitions import df_annotation_marker

In [2]:
    
dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
stat_df = pd.read_pickle(dataframe_dir)
stat_df

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian
EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03,,article 3\r\ndefinitions\r\nfor the purposes o...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
EU_32006L0066_Title_0_Chapter_0_Section_0_Article_25,,article 25\r\npenalties\r\nmember states shall...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,[],,,
EU_32009L0028_Title_0_Chapter_0_Section_0_Article_07,,article 7\r\njoint projects between member sta...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fabian]",[annotator:curation layer:Policydesigncharacte...,[],,,[],[annotator:Alisha layer:Policydesigncharacteri...,[annotator:Fabian layer:Policydesigncharacteri...
EU_32019L0944_Title_0_Chapter_6_Section_3_Article_47,,article 47\r\nindependence of the transmission...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,,,,[annotator:Alisha layer:Instrumenttypes type:I...,
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_14,,article 14\r\nadjustment of m0 and tm0 values\...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,,,,,[annotator:Fabian layer:Policydesigncharacteri...
...,...,...,...,...,...,...,...,...,...,...,...,...
EU_32018R1999_Title_0_Chapter_6_Section_3_Article_37,,article 37\r\nunion and national inventory sys...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[annotator:curation layer:Instrumenttypes type...,,[annotator:Fride layer:Instrumenttypes type:In...,,,[annotator:Alisha layer:Instrumenttypes type:I...,
EU_32006L0032_Title_0_Chapter_3_Section_0_Article_13,,article 13\r\nmetering and informative billing...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
EU_32009L0028_Title_0_Chapter_0_Section_0_Article_22,,article 22\r\nreporting by the member states\r...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fabian]",[annotator:curation layer:Instrumenttypes type...,[],,,,[annotator:Alisha layer:Instrumenttypes type:I...,[annotator:Fabian layer:Instrumenttypes type:I...
EU_32009R0079_Whereas,,whereas:\r\n(1)\r\nthe internal market compris...,"[start:0 stop:7 text:whereas tag_count:0, star...",ANNOTATION_IN_PROGRESS,"[Fabian, Onerva]",,[],,,,,[]


## Try the send coprus class

In [3]:
test_send_corp = Sent_Corpus(stat_df)
test_send_corp.df

,Sentence,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian
0,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,article 3\r\ndefinitions\r\nfor the purposes o...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
1,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,the payment for the services delivered is base...,"[start:2464 stop:2467 text:the tag_count:0, st...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Policydesigncharacteris...,,,,
2,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,that third party may or may not be an esco;\r\...,"[start:3321 stop:3325 text:that tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
3,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,this definition excludes electricity and natur...,"[start:4442 stop:4446 text:this tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
4,EU_32006L0066_Title_0_Chapter_0_Section_0_Arti...,article 25\r\npenalties\r\nmember states shall...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
4909,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,the commission shall also monitor the commodit...,"[start:635 stop:638 text:the tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,,,,
4910,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,2. the commission shall maintain a dialogue ...,"[start:811 stop:812 text:2 tag_count:0, start:...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
4911,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,"it shall, within that framework, pay particula...","[start:1119 stop:1121 text:it tag_count:0, sta...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Technologyandapplicatio...,,,,
4912,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,"3. in 2026, the commission shall submit, if ...","[start:1269 stop:1270 text:3 tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,


In [4]:
test_send_corp.df.shape

(4914, 12)

In [28]:
test_dir = repository.from_repository_name('EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03')
test_dir

policy:EU_32006L0032 title:Title_0 chapter:Chapter_1 section:Section_0 article:Article_03 sentence:None

In [29]:
len(test_send_corp.get_span_list(test_dir))

468

In [30]:
len(test_corp.get_span_list(test_dir))

468

## Test Sent Inter Annotator Agreement

In [31]:
test_send_evl = Sent_Inter_Annotator_Agreement(stat_df)

In [34]:
test_send_evl.df

,Sentence,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian
0,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,article 3\r\ndefinitions\r\nfor the purposes o...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
1,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,the payment for the services delivered is base...,"[start:2464 stop:2467 text:the tag_count:0, st...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Policydesigncharacteris...,,,,
2,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,that third party may or may not be an esco;\r\...,"[start:3321 stop:3325 text:that tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
3,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,this definition excludes electricity and natur...,"[start:4442 stop:4446 text:this tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
4,EU_32006L0066_Title_0_Chapter_0_Section_0_Arti...,article 25\r\npenalties\r\nmember states shall...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
4909,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,the commission shall also monitor the commodit...,"[start:635 stop:638 text:the tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Policydesigncharacteri...,[annotator:Fride layer:Policydesigncharacteris...,,,,
4910,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,2. the commission shall maintain a dialogue ...,"[start:811 stop:812 text:2 tag_count:0, start:...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Policydesigncharacte...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
4911,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,"it shall, within that framework, pay particula...","[start:1119 stop:1121 text:it tag_count:0, sta...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Technologyandapplicatio...,,,,
4912,EU_32018L2001_Title_0_Chapter_0_Section_0_Arti...,"3. in 2026, the commission shall submit, if ...","[start:1269 stop:1270 text:3 tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,


In [ ]:
scoring_metrics = ['f1_heuristic'] #, 'pygamma']
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic'] #, 'pygamma']
test_evaluator.keep_only_finished_articles()
test_evaluator.df.head()


In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [4]:
stat_df["Text"].loc['EU_32006L0066_Title_0_Chapter_0_Section_0_Article_25']

'article\xa025\r\npenalties\r\nmember states shall lay down rules on penalties applicable to infringements of national provisions adopted pursuant to this directive and shall take all necessary measures to ensure that they are implemented. the penalties provided for must be effective, proportionate and dissuasive. member\xa0states shall notify those measures to the commission by 26\xa0september 2008 and shall inform it without delay of any subsequent amendment to them.\r\n'

In [5]:
stat_df["Fride"].loc['EU_32006L0066_Title_0_Chapter_0_Section_0_Article_25'][1].text

'rules'

In [6]:
stat_df["Tokens"].loc['EU_32006L0066_Title_0_Chapter_0_Section_0_Article_25'][0]

start:0 stop:7 text:article tag_count:0

In [7]:
doc = nlp(stat_df["Text"].loc['EU_32006L0066_Title_0_Chapter_0_Section_0_Article_25'])
sentences = list(doc.sents)


In [8]:
nltk.download('punkt')
ERROR
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
text = "I do not like green eggs and ham. I do not like them Sam-I-am."

a_list = nltk.tokenize.sent_tokenize(text)

SyntaxError: invalid syntax (<ipython-input-8-0d92fa6581fc>, line 3)

In [5]:
#nltk.download('punkt')
text = stat_df["Text"].loc['EU_32019L0944_Title_0_Chapter_6_Section_3_Article_47']




In [9]:
doc = nlp(text)
sentences = list(doc.sents)

for sent in sentences:
    print(sent.text)
    print(text[text.index(sent.text):text.index(sent.text) + len(sent.text)] )
    print('-----')

article 47
independence of the transmission system operator
1.
article 47
independence of the transmission system operator
1.
-----
   without prejudice to the decisions of the supervisory body under article 49, the transmission system operator shall have:
(a)
effective decision-making rights, independent from the vertically integrated undertaking, with respect to assets necessary to operate, maintain or develop the transmission system; and
(b)
the power to raise money on the capital market in particular through borrowing and capital increase.
   without prejudice to the decisions of the supervisory body under article 49, the transmission system operator shall have:
(a)
effective decision-making rights, independent from the vertically integrated undertaking, with respect to assets necessary to operate, maintain or develop the transmission system; and
(b)
the power to raise money on the capital market in particular through borrowing and capital increase.
-----

2.

2.
-----
   the trans

In [8]:
import nltk
nltk.download('punkt')

sentences = nltk.tokenize.sent_tokenize(text)




marker = 0
for sent in sentences:
    print(sent)
    print(text[marker: marker+ len(sent)] )
    marker += len(sent)
    print('---------')

article 47
independence of the transmission system operator
1.   without prejudice to the decisions of the supervisory body under article 49, the transmission system operator shall have:
(a)
effective decision-making rights, independent from the vertically integrated undertaking, with respect to assets necessary to operate, maintain or develop the transmission system; and
(b)
the power to raise money on the capital market in particular through borrowing and capital increase.
article 47
independence of the transmission system operator
1.   without prejudice to the decisions of the supervisory body under article 49, the transmission system operator shall have:
(a)
effective decision-making rights, independent from the vertically integrated undertaking, with respect to assets necessary to operate, maintain or develop the transmission system; and
(b)
the power to raise money on the capital market in particular through borrowing and capital increase.
---------
2.   the transmission system o

[nltk_data] Downloading package punkt to /home/jkuettel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
from nltk.tokenize import sent_tokenize
text = "God is Great! I won a lottery."
type(sent_tokenize(text))

list

In [24]:
from nltk.data import load
language="english"

tokenizer = load(f"tokenizers/punkt/{language}.pickle")
text = "God is Great! I won a lottery."
sent_touples = tokenizer.span_tokenize(text, realign_boundaries = True)

for start,end in sent_touples:
    print(type(start), end)

<class 'int'> 13
<class 'int'> 30


In [23]:
i = 4
print(type(i))

<class 'int'>


questions:

dataframe for sentence level as other member or create child class
for repo, can adapt class such that sentence is other member?

In [14]:
import pandas as pd
stat_df_sentence = pd.DataFrame(columns = stat_df.columns)
stat_df_sentence = stat_df_sentence.rename(columns={'Policy':'Sentence'})


In [15]:
stat_df_sentence.head()

,Sentence,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian


In [16]:
def add_sent(obj, sentence):
    obj.rep.sentence = sentence
    return obj

In [17]:
from nltk.data import load
language="english"

tokenizer = load(f"tokenizers/punkt/{language}.pickle")

annotators = stat_df.columns[df_annotation_marker:]

## change: also add correct sentence for all the tags involved,
#finish add sentence method

for index, stat_df_row in stat_df.iterrows():
    
    raw_text = stat_df_row["Text"]

    tokens = stat_df_row["Tokens"]
    article_state = stat_df_row["Article_State"]
    finished_annotators = stat_df_row["Finished_Annotators"]
    cur_spans = stat_df_row["Curation"]
    
    sent_tuples = tokenizer.span_tokenize(raw_text, realign_boundaries = True)
    
    i = 0
    for start, stop in sent_tuples:
        sentence = 'Sentence_{}'.format(str(i))
        

        row = pd.Series(index=stat_df_sentence.columns, dtype = object)
        
        row["Sentence"] = stat_df_row.name +'_Sentence_{}'.format(str(i))
        row["Text"] = raw_text[start: stop]
        
        
        sent_tokens = [add_sent(tok,sentence) for tok in tokens if tok.start >= start and tok.stop <= stop]
        row["Tokens"] = sent_tokens
            
        row["Article_State"] = article_state
        row["Finished_Annotators"] = finished_annotators
        
        if type(cur_spans) == list:
            row["Curation"]= [spn for spn in cur_spans if spn.start >= start and spn.stop <= stop]
        else:
            row["Curation"] = ''
        
        for annotator in annotators:
            
            if annotator in finished_annotators and type(stat_df_row[annotator]) == list:
                row[annotator] = [add_sent(spn,sentence) for spn in stat_df_row[annotator] if spn.start >= start and spn.stop <= stop]
                
            else:
                row[annotator] = ''
            
        stat_df_sentence = stat_df_sentence.append(row, ignore_index=True)
        i +=1



In [18]:
len(list(chain.from_iterable(stat_df['Curation'])))

0       [start:0 stop:7 text:article tag_count:0, star...
1       [start:2464 stop:2467 text:the tag_count:0, st...
2       [start:3321 stop:3325 text:that tag_count:0, s...
3       [start:4442 stop:4446 text:this tag_count:0, s...
4       [start:0 stop:7 text:article tag_count:0, star...
                              ...                        
4909    [start:635 stop:638 text:the tag_count:0, star...
4910    [start:811 stop:812 text:2 tag_count:0, start:...
4911    [start:1119 stop:1121 text:it tag_count:0, sta...
4912    [start:1269 stop:1270 text:3 tag_count:0, star...
4913    [start:1693 stop:1694 text:4 tag_count:0, star...
Name: Tokens, Length: 4914, dtype: object

In [ ]:
len(list(chain.from_iterable(stat_df_sentence["Tokens"])))

In [13]:
def add_sent(tok):
    tok.rep.sentence = 'test sentence'
    return tok

In [14]:
stat_df_sentence.iloc[1]

Sentence               EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...
Text                   the payment for the services delivered is base...
Tokens                                                                []
Article_State                                          CURATION_FINISHED
Finished_Annotators                                      [Fride, Onerva]
Curation               [annotator:curation layer:Instrumenttypes type...
Onerva                 [annotator:Onerva layer:Instrumenttypes type:I...
Fride                  [annotator:Fride layer:Policydesigncharacteris...
Lynn                                                                    
Sebastian                                                               
Alisha                                                                  
Fabian                                                                  
Name: 1, dtype: object

In [15]:
map(add_sent, stat_df.iloc[1]["Tokens"])

In [15]:
stat_df_sentence.head()

,Sentence,Text,Tokens,Article_State,Finished_Annotators,Curation,Onerva,Fride,Lynn,Sebastian,Alisha,Fabian
0,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,article 3\r\ndefinitions\r\nfor the purposes o...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
1,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,the payment for the services delivered is base...,"[start:2464 stop:2467 text:the tag_count:0, st...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Policydesigncharacteris...,,,,
2,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,that third party may or may not be an esco;\r\...,"[start:3321 stop:3325 text:that tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
3,EU_32006L0032_Title_0_Chapter_1_Section_0_Arti...,this definition excludes electricity and natur...,"[start:4442 stop:4446 text:this tag_count:0, s...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,
4,EU_32006L0066_Title_0_Chapter_0_Section_0_Arti...,article 25\r\npenalties\r\nmember states shall...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[annotator:curation layer:Instrumenttypes type...,[annotator:Onerva layer:Instrumenttypes type:I...,[annotator:Fride layer:Instrumenttypes type:In...,,,,


In [31]:
stat_df.iloc[0]

Policy                                                                  
Text                   article 3\r\ndefinitions\r\nfor the purposes o...
Tokens                 [start:0 stop:7 text:article tag_count:0, star...
Article_State                                          CURATION_FINISHED
Finished_Annotators                                      [Fride, Onerva]
Curation               [annotator:curation layer:Instrumenttypes type...
Onerva                 [annotator:Onerva layer:Instrumenttypes type:I...
Fride                  [annotator:Fride layer:Instrumenttypes type:In...
Lynn                                                                    
Sebastian                                                               
Alisha                                                                  
Fabian                                                                  
Name: EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03, dtype: object

In [10]:
len(list(chain.from_iterable(stat_df['Onerva'])))

9627

In [11]:
len(list(chain.from_iterable(stat_df_sentence['Onerva'])))

9528

In [8]:
len(list(chain.from_iterable(stat_df_sentence['Curation'])))

20310

In [9]:
len(list(chain.from_iterable(stat_df['Curation'])))

20310

In [48]:
hello = hello.append(pd.Series(['ma','se'], index=hello.columns ), ignore_index=True)

In [49]:
hello

,he,se
0,ma,se


In [51]:
hello = hello.append(pd.Series([_ ,_ ], index=hello.columns ), ignore_index=True)

In [52]:
hello

,he,se
0,ma,se
1,he se 0 ma se,he se 0 ma se


In [53]:
hello = hello.append(pd.Series([_ ,_ ], index=hello.columns ), ignore_index=True)

In [54]:
hello

,he,se
0,ma,se
1,he se 0 ma se,he se 0 ma se
2,he se 0 ...,he se 0 ...


In [34]:
class animal:
    
    def __init__(self, name, weight, size):
        self.name = name
        self.weight = weight
        self.size = size
        
    def get_bmi(self):
        return self.weight * self.size

class lion(animal):
    pass
    


In [36]:
li = lion('peter', 100, 2)
li.get_bmi()

200

In [32]:
class animal:
    
    def __init__(self, name, weight, size):
        self.name = name
        self.weight = weight
        self.size = size
        
    def get_bmi(self):
        return self.weight * self.size

class lion(animal):
    
    def __init__(self, name, weight, size, age):
        
        super().__init__(name, weight, size)
        self.age = age
    
    def get_bmi(self):
        bmi = super().get_bmi()
        return self.age * bmi
        
        
    


In [19]:
li = lion('peter', 100, 2, 33)

In [5]:
test = pd.DataFrame(columns = ['one', 'two'])

In [11]:
cols = list(test.columns)
cols.append("Sentence_Start")
cols.append("Sentence_Stop")